# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [222]:
import warnings
warnings.filterwarnings('ignore')

In [223]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [224]:
#!pip install pandas

In [225]:
import pandas as pd
df = pd.read_csv('data\Data.csv')

In [226]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [227]:
# !pip install langchain_community

In [228]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [229]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [230]:
prompt = ChatPromptTemplate.from_template("Write a review for a product called {product}")





In [231]:

chain = LLMChain(llm=llm, prompt=prompt)

In [232]:
product = "Learn Python in 3 Days Book"
response = chain.run(product=product)

## SimpleSequentialChain

In [233]:
from langchain.chains import SimpleSequentialChain

In [234]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   " Write a review for a product called {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [235]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a positive review for {product}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [236]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [237]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
I recently purchased the "Learn Python in 3 Days Book" and I am extremely happy with my purchase. This book is perfect for beginners who are looking to learn Python quickly and efficiently. The step-by-step instructions are easy to follow and the examples provided are very helpful in understanding the concepts.

I love how the book is broken down into 3 days, making it easy to digest the information and practice each day's lesson. The exercises at the end of each chapter really helped solidify my understanding of the material.

Overall, I highly recommend the "Learn Python in 3 Days Book" to anyone looking to learn Python in a short amount of time. It's a great resource for beginners and I have already seen improvements in my programming skills after just a few days of working through the book. It's definitely worth the investment!
I am really impressed with the "Learn Python in 3 Days Book". The clear and concise explanations make it eas

'I am really impressed with the "Learn Python in 3 Days Book". The clear and concise explanations make it easy for beginners to grasp the concepts and the exercises are perfect for reinforcing what you have learned. I love how the book is structured into 3 days, making it manageable to learn Python quickly and efficiently. I highly recommend this book to anyone looking to learn Python in a short amount of time. It\'s a fantastic resource that has already helped me improve my programming skills. Definitely worth the purchase!'

**Repeat the above twice for different products**

## SequentialChain

In [238]:
from langchain.chains import SequentialChain

In [239]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  " Write a review for a product called {product}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Review" #Give a name to your output
                    )


In [240]:
second_prompt = ChatPromptTemplate.from_template(
  "Write a summary of 3 features of the product based on this review: {Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="Summary")


In [241]:
third_prompt = ChatPromptTemplate.from_template(
  "Translate this review to Arabic: {Review}"
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="Arabic_Review")


In [242]:
fourth_prompt = ChatPromptTemplate.from_template(
  "Based on the summary: '{Summary}' and the Arabic review: '{Arabic_Review}', write a follow-up message to the customer."
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="Final_Message")


In [243]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["product"],
    output_variables=["Review", "Summary", "Arabic_Review", "Final_Message"],
    verbose=True
)


In [244]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Review': "Je suis très déçu par ce produit. Je trouve le goût médiocre et la mousse ne tient pas du tout, c'est vraiment bizarre. J'achète habituellement les mêmes produits dans le commerce et le goût est bien meilleur. Je me demande si ce n'est pas un vieux lot ou une contrefaçon. Je ne recommande pas du tout ce produit.",
 'Summary': '1. The taste of the product is described as mediocre and not enjoyable.\n2. The foam does not hold up well, as it quickly disappears after being poured.\n3. The reviewer questions if the product is possibly old or a counterfeit, as they have had better experiences with similar products from other brands. Overall, the reviewer does not recommend this product.',
 'Arabic_Review': 'أنا خائب جدًا من هذا المنتج. أجده ذو طعم سيء والرغوة لا تستمر على الإطلاق، إنه غريب حقًا. عادةً ما

**Repeat the above twice for different products or reviews**

## Router Chain

In [245]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
What is Newton's second law of motion?
{input}
"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
What is the derivative of sin(x)?
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
How did the Industrial Revolution change society in Europe? 
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
How does a binary search algorithm work?
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
What is DNA and what is its function in the cell?
{input}"""

VideoGames_template = """You are an excellent video game developer and designer. \
You have a deep understanding of game mechanics, storytelling, player engagement, \
and game development tools and engines. \
You are skilled at observing trends in the gaming industry, analyzing player feedback, \
and designing immersive and balanced gameplay experiences. \
You can clearly explain how different elements like game mechanics, level design, graphics, \
and sound work together to create an engaging game. \
You make connections between gameplay theory and practical development \
to answer challenging questions and offer valuable advice on game creation and design.

Here is a question:
What are the popular game development environments and their popular games?
{input}"""

In [246]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    },
    {
         "name": "VideoGames",
         "description": "Good for answering VideoGames questions",
         "prompt_template": VideoGames_template
    }

]

In [247]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [248]:
llm = ChatOpenAI(temperature=0)

In [249]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [250]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [251]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": VideoGames"
    "next_inputs": Good for answering VideoGames questions"
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [252]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [253]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [254]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Newton's second law of motion states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. In equation form, it is written as F = ma, where F is the net force, m is the mass of the object, and a is the acceleration.\n\nBlack body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the body."

In [255]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The derivative of sin(x) is cos(x).\n\n2 + 2 = 4'

In [256]:
chain.run("what is 500 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 500 + 2'}
> Finished chain.


'The derivative of sin(x) is cos(x).\n\n500 + 2 = 502'



> Entering new MultiPromptChain chain...
VideoGames: {'input': 'what is ps5 ?'}
> Finished chain.


'Some popular game development environments include Unity, Unreal Engine, GameMaker Studio, and Godot Engine. These environments are used by game developers to create a wide range of popular games across various genres.\n\nSome popular games created using Unity include "Hollow Knight," "Cuphead," and "Ori and the Blind Forest." Unreal Engine has been used to create games like "Fortnite," "Gears of War," and "BioShock Infinite." GameMaker Studio is known for games like "Hyper Light Drifter," "Hotline Miami," and "Undertale." Godot Engine has been used for games like "Hollow Knight: Silksong," "Heartbound," and "MoonQuest."\n\nAs for the PS5, it is the latest gaming console released by Sony Interactive Entertainment. It is the successor to the PS4 and features improved hardware capabilities, faster load times, ray tracing support, and a new controller called the DualSense. The PS5 has a growing library of games, including titles like "Demon\'s Souls," "Spider-Man: Miles Morales," and "Ra

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [258]:
chain.run("what is ps5 ?")



> Entering new MultiPromptChain chain...
VideoGames: {'input': 'what is ps5 ?'}
> Finished chain.


'Some popular game development environments include Unity, Unreal Engine, GameMaker Studio, and Godot Engine. \n\nUnity is known for its versatility and ease of use, making it a popular choice for indie developers and large studios alike. Some popular games developed with Unity include "Hollow Knight," "Cuphead," and "Ori and the Blind Forest."\n\nUnreal Engine is known for its powerful graphics capabilities and is often used for creating AAA titles. Some popular games developed with Unreal Engine include "Fortnite," "Gears of War," and "Borderlands."\n\nGameMaker Studio is popular for its simplicity and accessibility, making it a great choice for beginners. Some popular games developed with GameMaker Studio include "Undertale," "Hyper Light Drifter," and "Hotline Miami."\n\nGodot Engine is an open-source game development environment known for its flexibility and community support. Some popular games developed with Godot Engine include "Hollow Knight: Silksong," "Heartbeat," and "Gibbo